We'll work with a dataset on sold houses in Ames, Iowa. Each row in the dataset describes the properties of a single house as well as the amount it was sold for. In this course, we'll build models that predict the final sale price from its other attributes. Specifically, we'll explore the following questions:

* Which properties of a house most affect the final sale price?
* How effectively can we predict the sale price from just its properties?

Here are some of the columns:

 * Lot Area: Lot size in square feet.
 * Overall Qual: Rates the overall material and finish of the house.
 * Overall Cond: Rates the overall condition of the house.
 * Year Built: Original construction date.
 * Low Qual Fin SF: Low quality finished square feet (all floors).
 * Full Bath: Full bathrooms above grade.
 * Fireplaces: Number of fireplaces.
 * Let's start by generating train and test datasets and getting more familiar with the data.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

In [4]:
df = pd.read_csv('AmesHousing.txt', delimiter='\t')

In [8]:
def transform_features(df):
    return df

def select_feature(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    preds = lr.predict(test[features])
    rmse = np.sqrt(mean_squared_error(test['SalePrice'], preds))
    
    return rmse

transform_df = transform_features(df)
filtered_df = select_feature(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.25161263909

# Feature Engineering

Let's now start removing features with many missing values, diving deeper into potential categorical features, and transforming text and numerical columns. Update transform_features() so that any column from the data frame with more than 25% (or another cutoff value) missing values is dropped. We also need to remove any columns that leak information about the sale (e.g. like the year the sale happened). In general, the goal of this function is to:

 * remove features that we don't want to use in the model, just based on the number of missing values or data leakage
 * transform features into the proper format (numerical to categorical, scaling numerical, filling in missing values, etc)
 * create new features by combining other features

Here are some of the questions that will serve as our guideline during the machine learning process:

* Which columns contain less than 5% missing values?
 * For numerical columns that meet this criteria, let's fill in the missing values using the most popular value for that column.
* What new features can we create, that better capture the information in some of the features?
 * An example of this would be the years_until_remod feature we created in the last mission.
*Which columns need to be dropped for other reasons?
 * Which columns aren't useful for machine learning?
 * Which columns leak data about the final sale?

In [18]:
null_cols = df.isnull().mean()*100

# drop columns with 5% or more missing values for now
df1 = df[null_cols[null_cols < 5].index]

In [23]:
# drop any text columns with 1 or more missing values for now
text_mv_counts = df1.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

# filter series to columns containing any missing values
null_cols_2 = text_mv_counts[text_mv_counts > 0]

df1 = df1.drop(null_cols_2.index, 1)

In [29]:
# for numerical columns wtih missing values, fill in with most common values
num_mv_counts = df1.select_dtypes(include=['integer', 'float']).isnull().sum()
filled_num_mv = num_mv_counts[num_mv_counts > 0].sort_values()

replacement_values = df1[filled_num_mv.index].mode().to_dict(orient='records')[0]
replacement_values

{'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Bsmt Unf SF': 0.0,
 'Total Bsmt SF': 0.0,
 'Garage Cars': 2.0,
 'Garage Area': 0.0,
 'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Mas Vnr Area': 0.0}

In [30]:
df1 = df1.fillna(replacement_values)

Order              0
PID                0
MS SubClass        0
MS Zoning          0
Lot Area           0
Street             0
Lot Shape          0
Land Contour       0
Utilities          0
Lot Config         0
Land Slope         0
Neighborhood       0
Condition 1        0
Condition 2        0
Bldg Type          0
House Style        0
Overall Qual       0
Overall Cond       0
Year Built         0
Year Remod/Add     0
Roof Style         0
Roof Matl          0
Exterior 1st       0
Exterior 2nd       0
Mas Vnr Area       0
Exter Qual         0
Exter Cond         0
Foundation         0
BsmtFin SF 1       0
BsmtFin SF 2       0
                  ..
Central Air        0
1st Flr SF         0
2nd Flr SF         0
Low Qual Fin SF    0
Gr Liv Area        0
Bsmt Full Bath     0
Bsmt Half Bath     0
Full Bath          0
Half Bath          0
Bedroom AbvGr      0
Kitchen AbvGr      0
Kitchen Qual       0
TotRms AbvGrd      0
Functional         0
Fireplaces         0
Garage Cars        0
Garage Area  

In [31]:
df1.isnull().sum().value_counts()

0    64
dtype: int64

**Creating new feature**

In [33]:
years_sold = df1['Yr Sold'] - df1['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [34]:
years_since_remod = df1['Yr Sold'] - df1['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [35]:
# create new columns 
df1['years_sold'] = years_sold
df1['years_since_remod'] = years_since_remod

# drop rows with negative values
df1 = df1.drop([1702, 2180, 2181], axis=0)

# drop orginal columns
df1 = df1.drop(['Year Built', 'Year Remod/Add'], axis=1)

**Dropping columns that:**
 * Ain't useful for ML
 * Leaking data about the final sale

In [36]:
# drop cols that ain't useful for ML
df1 = df1.drop(['PID', 'Order'],1)

# drop cols that leak info
df1 = df1.drop(['Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'],1)

In [38]:
def transform_features(df):
    num_missing = df.isnull().mean()*100
    df = df[num_missing[num_missing < 5].index] 
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    null_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(null_cols_2.index, 1)
    
    num_mv_counts = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    filled_num_mv = num_mv_counts[num_mv_counts > 0].sort_values()
    replacement_values = df[filled_num_mv.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['years_sold'] = years_sold
    df['years_since_remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)
    
    df = df.drop(['Year Built', 'Year Remod/Add', 'PID', 'Order', 'Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis=1)
    
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer','float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    preds = lr.predict(test[features])
    rmse = np.sqrt(mean_squared_error(test['SalePrice'], preds))
    
    return rmse

df = pd.read_csv('AmesHousing.txt', delimiter='\t')
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.367312413066

## Feature Selection

* Generate a correlation heatmap matrix of the numerical features in the training data set.
 * Which features correlate strongly with our target column, SalePrice?
 * Calculate the correlation coefficients for the columns that seem to correlate well with SalePrice. Because we have a pipeline in place, it's easy to try different features and see which features result in a better cross validation score.

* Which columns in the data frame should be converted to the categorical data type? All of the columns that can be categorized as nominal variables are candidates for being converted to categorical. Here are some other things you should think about:
 * If a categorical column has hundreds of unique values (or categories), should you keep it? When you dummy code this column, hundreds of columns will need to be added back to the data frame.
 * Which categorical columns have a few unique values but more than 95% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).

* Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

* What are some ways we can explore which categorical columns "correlate" well with SalePrice?

* Update the logic for the `select_features()` function. This function should take in the new, modified train and test data frames that were returned from `transform_features()`.

In [39]:
numerical_df = transform_df.select_dtypes(include=['integer', 'float'])
numerical_df

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,years_sold,years_since_remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12
5,60,9978,6,6,20.0,602.0,0.0,324.0,926.0,926,...,360,36,0,0,0,0,0,195500,12,12
6,120,4920,8,5,0.0,616.0,0.0,722.0,1338.0,1338,...,0,0,170,0,0,0,0,213500,9,9
7,120,5005,8,5,0.0,263.0,0.0,1017.0,1280.0,1280,...,0,82,0,0,144,0,0,191500,18,18
8,120,5389,8,5,0.0,1180.0,0.0,415.0,1595.0,1616,...,237,152,0,0,0,0,0,236500,15,14
9,60,7500,7,5,0.0,0.0,0.0,994.0,994.0,1028,...,140,60,0,0,0,0,0,189000,11,11


In [40]:
abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
years_since_remod    0.534985
Full Bath            0.546118
years_sold           0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: Sale

In [41]:
# keep columns with coef > 0.4 (arbitrarily for now)
abs_corr_coeffs[abs_corr_coeffs > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
years_since_remod    0.534985
Full Bath            0.546118
years_sold           0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [42]:
# drop cols with less than 0.4 coef with SalePrice
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, 1)

**Which categorical columns to keep?**

In [43]:
# list of cols that are meant to be categorical
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

* Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?

* If a categorical column has hundreds of unique values (or categories), should we keep it? When we dummy code this column, hundreds of columns will need to be added back to the data frame.

In [49]:
df1['Overall Qual'].value_counts()

5     825
6     732
7     602
8     349
4     226
9     107
3      40
10     29
2      13
1       4
Name: Overall Qual, dtype: int64

In [50]:
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

# count unique values in categorical cols
unique_counts = transform_df[transform_cat_cols].apply(lambda x: len(x.value_counts())).sort_values()

# arbitrarily cut off 10 unique values (for now)
drop_unique_cols = unique_counts[unique_counts > 10].index
transform_df = transform_df.drop(drop_unique_cols, 1)

In [53]:
# select the remaining text cols and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')

# create dummy cols and add back to dataframe
transform_df = pd.concat([transform_df, pd.get_dummies(transform_df.select_dtypes(include=['category']))],1).drop(text_cols, 1)

**Update select_features function**

In [68]:
def transform_features(df):
    num_missing = df.isnull().mean()*100
    df = df[num_missing[num_missing < 5].index] 
    
    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    null_cols_2 = text_mv_counts[text_mv_counts > 0]
    df = df.drop(null_cols_2.index, 1)
    
    num_mv_counts = df.select_dtypes(include=['integer', 'float']).isnull().sum()
    filled_num_mv = num_mv_counts[num_mv_counts > 0].sort_values()
    replacement_values = df[filled_num_mv.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values)
    
    years_sold = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['years_sold'] = years_sold
    df['years_since_remod'] = years_since_remod
    df = df.drop([1702, 2180, 2181], axis=0)
    
    df = df.drop(['Year Built', 'Year Remod/Add', 'PID', 'Order', 'Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis=1)
    
    return df

def select_features(df):
    numerical_df = transform_df.select_dtypes(include=['integer', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, 1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)
            
    unique_counts = df[transform_cat_cols].apply(lambda x: len(x.value_counts())).sort_values()
    drop_unique_cols = unique_counts[unique_counts > 10].index
    df = df.drop(drop_unique_cols, 1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))],1).drop(text_cols, 1)  
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer','float'])
    features = numeric_df.columns.drop('SalePrice')
    lr = linear_model.LinearRegression()
    
    if k==0:
        train = df[:1460]
        test = df[1460:]
        lr.fit(train[features], train['SalePrice'])
        preds = lr.predict(test[features])
        rmse = np.sqrt(mean_squared_error(test['SalePrice'], preds))
        return rmse 
    
    if k==1:
        # randomize all rows (frac=1) and return
        shuffled_df = df.sample(frac=1,)
        train = df[:1460]
        test = df[1460:]
        lr.fit(train[features], train['SalePrice'])
        preds_1 = lr.predict(test[features])
        rmse_1 = np.sqrt(mean_squared_error(test['SalePrice'], preds_1))
        print(rmse_1)
        lr.fit(test[features], test['SalePrice'])
        preds_2 = lr.predict(train[features])
        rmse_2 = np.sqrt(mean_squared_error(train['SalePrice'], preds_2))
        print(rmse_2)
        avg_rmse = np.mean([rmse_1, rmse_2])
        return avg_rmse 
    
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train['SalePrice'])
            preds = lr.predict(test[features])
            rmse = np.sqrt(mean_squared_error(test['SalePrice'], preds))
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse

df = pd.read_csv('AmesHousing.txt', delimiter='\t')
transformed_df = transform_features(df)
filtered_df = select_features(transformed_df)
rmse = train_and_test(filtered_df, k=15)

rmse

[25918.19825099534, 25646.925840883094, 27431.933941359977, 28185.099822143013, 25439.106220537913, 29847.427262701804, 54481.12068629827, 28098.449031916734, 26069.63328938557, 22758.63565019265, 28935.420032246686, 24066.799563474746, 26933.97178846003, 24322.552086229214, 29139.488751100944]


28484.984147861735